In [2]:
import json
import random
import torch
import torchaudio
import librosa
import pandas as pd
import numpy as np

from IPython.display import display, HTML

from datasets import ClassLabel
from datasets import load_dataset, load_metric
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2Processor, Wav2Vec2FeatureExtractor, Wav2Vec2ForCTC, Trainer, TrainingArguments

/home/scutum/envs/data-science/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [1]:
common_voice_train = load_dataset("common_voice", "tr", split="train+validation")
common_voice_test = load_dataset("common_voice", "tr", split="test")

NameError: name 'load_dataset' is not defined

In [3]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [4]:
common_voice_train.column_names

NameError: name 'common_voice_train' is not defined

In [ ]:
common_voice_train = common_voice_train.remove_columns(column_names=['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'segment', 'up_votes'])
common_voice_test = common_voice_test.remove_columns(column_names=['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'segment', 'up_votes'])

In [7]:
show_random_elements(common_voice_train)

,path,sentence
0,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_18731995.mp3,Ya tahmin hatalı çıkarsa?
1,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_19170901.mp3,Komisyon çifte vatandaşlığı da tartışıyor.
2,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_17427247.mp3,Ardından kilisede evlenme töreni yapılıyor.
3,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_18756243.mp3,Türkiye tutuklamaları memnuniyetle karşıladı.
4,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_18169121.mp3,Batı Balkanlar nereye doğru gidiyor?
5,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_17348439.mp3,Şirket bitmiş ürünlerini satmakta başarılı.
6,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_17953928.mp3,Ancak Nikoliç devam etmekte kararlı.
7,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_17559437.mp3,"""Makedonya üyelik başvurusunu yirmi iki Mart'ta sundu."""
8,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_21317254.mp3,Sergide yüz elli taslak ve kolaj çalışması yer alıyor.
9,/home/scutum/.cache/huggingface/datasets/downloads/extracted/56b9d11bf987fa5ac213dcaa581cb6dfe16abd5b7094815f0a07ae261121f12c/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_17341280.mp3,Bunu kanıtlayacak pek fazla delil yok.


In [8]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [10]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-f62dcd4947ab6451.arrow
Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-92d876638c917d70.arrow


In [11]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [12]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

In [13]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test['vocab'][0]))

In [14]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

In [16]:
vocab_dict['|'] = vocab_dict[' ']

In [18]:
del vocab_dict[' ']

In [19]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

40

In [20]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [21]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [22]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("path/to/folder")

***
***

### Load a custom audio file. // Trial

In [21]:
sa, sr = torchaudio.load('/home/scutum/covost_tr/tr/raw/clips/common_voice_tr_20210689.mp3')

In [22]:
sa.shape, sr

(torch.Size([1, 161280]), 48000)

In [23]:
sa

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.8120e-05,
         -3.4273e-07,  2.7820e-05]])

In [24]:
type(sa)

torch.Tensor

In [17]:
np.asarray(sa.reshape(-1)).shape

(161280,)

In [18]:
sa = librosa.resample(np.asarray(sa.reshape(-1)), 44_100, 16_000)

In [20]:
sa.shape

(58515,)

***
***

In [26]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [27]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)

Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-c3e35f3c1c73eac0.arrow


In [28]:
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-bdb2cb17a36f2336.arrow


In [29]:
def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [30]:
common_voice_train = common_voice_train.map(resample, num_proc=4)
common_voice_test = common_voice_test.map(resample, num_proc=4)

Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-5d2f8e9dc351e062.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-affae8bd42bd86b6.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-ad01a5a6695b7b76.arrow
Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-f107ca79b733d04a.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-bdf52846ed042bea.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-e8e222cad49b1b63.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-02f552ad1c408a56.arrow
Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-47aebb1244894886.arrow


In [31]:
common_voice_train.column_names

['sampling_rate', 'speech', 'target_text']

In [32]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: oysa bu on yıl sonra gerçekleşti 
Input array shape: (54144,)
Sampling rate: 16000


In [33]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [44]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-1a19bf256dd9b52d.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-dcfa5d153ece4398.arrow
Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-a8561f4f748db8c2.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-bd31e74c1d66747b.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-d8f22c64a92e0180.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-ad8e5267b8c23cdd.arrow
Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-c96682cdd57de64e.arrow


Loading cached processed dataset at /home/scutum/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-195da893bc377574.arrow


In [45]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [46]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [47]:
wer_metric = load_metric("wer")

In [48]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [ ]:
model.freeze_feature_extractor()

In [ ]:
training_args = TrainingArguments(
  output_dir="/home/scutum/wav2vec2-large-xlsr-turkish-demo",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=4,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=1,
)

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [5]:
model = Wav2Vec2ForCTC.from_pretrained("path/to/saved_model").to("cuda")

In [6]:
processor = Wav2Vec2Processor.from_pretrained("path/to/saved_processor")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [49]:
input_dict = processor(common_voice_test["input_values"][2], return_tensors="pt", padding=True)

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


***
***

In [33]:
sa = processor(sa, return_tensors='pt', padding=True)

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [34]:
sa

{'input_values': tensor([[ 0.0064,  0.0064,  0.0064,  ..., -0.0042,  0.0059,  0.0264]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [35]:
logits = model(sa.input_values.to("cuda")).logits

In [36]:
pred_ids = torch.argmax(logits, dim=-1)[0]

In [37]:
print(processor.decode(pred_ids))

ışıkları açar mısın


***
***

In [ ]:
logits = model(input_dict.input_values.to("cuda")).logits

In [ ]:
pred_ids = torch.argmax(logits, dim=-1)[0]

In [ ]:
common_voice_test_transcription = load_dataset("common_voice", "tr", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[2]["sentence"].lower())